In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "YourSecurePassword1234!"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('AnimalRescueDashboard')

app.layout = html.Div([
    # Logo + Title
    html.Center([
        html.A(
            html.Img(src='assets/Grazioso Salvare Logo.png', style={'width':'150px', 'marginBottom':'10px'}),
            href='https://www.snhu.edu'
        ),
        html.H1('Grazioso Salvare Animal Rescue Dashboard')
    ]),
    html.Hr(),


    # Filter buttons
    html.Div([
        html.Button("Water Rescue", id="btn-water", n_clicks=0),
        html.Button("Mountain Rescue", id="btn-mountain", n_clicks=0),
        html.Button("Disaster Rescue", id="btn-disaster", n_clicks=0),
        html.Button("Reset", id="btn-reset", n_clicks=0)
    ], style={"textAlign": "center", "margin": "10px"}),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Div(id='map-id'),
    html.Br(),
    dcc.Graph(id='pie-chart'),

    # Identifier footer
    html.Div(
        "Jeremy Brown | SNHU CS-340 MongoDB Dashboard",
        style={"marginTop": 20, "fontSize": 14, "color": "gray", "textAlign": "center"}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter button callback
@app.callback(
    Output('datatable-id', 'data'),
    [Input('btn-water', 'n_clicks'),
     Input('btn-mountain', 'n_clicks'),
     Input('btn-disaster', 'n_clicks'),
     Input('btn-reset', 'n_clicks')]
)
def update_table(water, mountain, disaster, reset):
    ctx = dash.callback_context

    if not ctx.triggered:
        dff = df
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'btn-water':
            dff = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed": "Labrador Retriever Mix"}))
        elif button_id == 'btn-mountain':
            dff = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed": "German Shepherd"}))
        elif button_id == 'btn-disaster':
            dff = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed": "Doberman Pinscher"}))
        else:
            dff = df

    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    return dff.to_dict('records')


# Update map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or not index:
        return [html.Div("No data available")]

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    try:
        lat = dff.at[row, 'location_lat']
        lon = dff.at[row, 'location_long']
        breed = dff.at[row, 'breed'] if 'breed' in dff.columns else ''
        name = dff.at[row, 'name'] if 'name' in dff.columns else ''
    except Exception:
        return [html.Div("Invalid or missing location data")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
                   dl.TileLayer(id='base-layer-id'),
                   dl.Marker(position=[lat, lon], children=[
                       dl.Tooltip(breed),
                       dl.Popup([
                           html.H1("Animal Name"),
                           html.P(name)
                       ])
                   ])
               ])
    ]


# Pie chart callback
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff.columns or dff.empty:
        return px.pie(values=[1], names=["No Data"], title="Breed Distribution")

    fig = px.pie(
        dff,
        names='breed',
        title="Breed Distribution",
        width=800,   # wider figure
        height=800   # taller figure
    )
    fig.update_layout(
        legend=dict(
            orientation="v",
            x=1.05,
            y=0.5,
            font=dict(size=12),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    return fig

# Button style callback
@app.callback(
    Output('btn-water', 'style'),
    Output('btn-mountain', 'style'),
    Output('btn-disaster', 'style'),
    Output('btn-reset', 'style'),
    [Input('btn-water', 'n_clicks'),
     Input('btn-mountain', 'n_clicks'),
     Input('btn-disaster', 'n_clicks'),
     Input('btn-reset', 'n_clicks')]
)
def update_button_styles(water, mountain, disaster, reset):
    ctx = dash.callback_context

    default_style = {"margin": "5px", "backgroundColor": "#f0f0f0"}
    active_style = {"margin": "5px", "backgroundColor": "#555", "color": "white"}

    if not ctx.triggered:
        return default_style, default_style, default_style, default_style
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        return (
            active_style if button_id == 'btn-water' else default_style,
            active_style if button_id == 'btn-mountain' else default_style,
            active_style if button_id == 'btn-disaster' else default_style,
            active_style if button_id == 'btn-reset' else default_style
        )

app.run_server(debug=True)

Dash app running on http://127.0.0.1:15134/
